In [ ]:
#Make separate data files for each lineage

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from augur.utils import json_to_tree
import json

In [15]:
#need to look at tree json file to find the branch name for the common ancestor of the different lineages
#format of dict {lineage:common_ancestor}


subgenotype_ancestors = {'A2': 'NODE_0000405', 'D3':'NODE_0001139'}

In [16]:
for subgenotype, ca in subgenotype_ancestors.items():
    genotype = subgenotype[0]
    #read in tree
    with open(f'../auspice/hepatitisB_{genotype}.json', 'r') as f:
        tree_json = json.load(f)

    #Put tree in Bio.Phylo format
    tree = json_to_tree(tree_json)
    
    #store all tip names for each subgenotype
    tips_in_this_subgenotype = []

    
    #find tips descending from common ancestor branch of the lineages
    for node in tree.find_clades(terminal=True):
        parents = [x.name for x in tree.get_path(node)]
        if ca in parents:
            tips_in_this_subgenotype.append(node.name)
            
    seq_records = []

    #exclude samples that are called a different subgenotype in the genbank file
    for record in SeqIO.parse(open(f"hepatitisB_{genotype}.fasta","r"), "fasta"):
        accession = record.description.split('|')[0]
        #check for the tips that cluster with this subgenotype
        if accession in tips_in_this_subgenotype:
            sub = record.description.split('|')[6]
            #find whether they have a different annotation
            if sub == subgenotype or sub == 'None':
                seq_records.append(SeqRecord(record.seq, id=record.description, description=''))
        
        
    SeqIO.write(seq_records, f'hepatitisB_{subgenotype}.fasta', "fasta")